# Code to check that the initial and final geometries are not too different

In [1]:
import os
import pickle
import numpy as np
from ase import io
from collections import defaultdict

In [2]:
def geom_is_good(initial_structure, final_structure):
    """
    Args:
    initial_structure: starting structure for an optimization
    final_structure: final structure from optimization
    Returns:
    True if no problems found
    False otherwise
    """
    atoms_obj = io.read(initial_structure)
    init_struct = atoms_obj.get_positions()
    final_struct = io.read(final_structure).get_positions()
    atomic_nums = atoms_obj.get_atomic_numbers()
    # set the metal positions as the origin
    origin_i = init_struct[np.argmax(atomic_nums)]
    origin_f = final_struct[np.argmax(atomic_nums)]
    init_distances = []
    final_distances = []
    for xyz_i, xyz_f in zip(init_struct, final_struct):
        init_distances.append(np.sqrt((xyz_i[0]-origin_i[0])**2+(xyz_i[1]-origin_i[1])**2+(xyz_i[2]-origin_i[2])**2))
        final_distances.append(np.sqrt((xyz_f[0]-origin_f[0])**2+(xyz_f[1]-origin_f[1])**2+(xyz_f[2]-origin_f[2])**2))
        
    num_neighbours = 6
    nearest_neighbours_i = np.argpartition(init_distances, num_neighbours)
    nearest_neighbours_i = [nn for nn in nearest_neighbours_i[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    nearest_neighbours_f = np.argpartition(final_distances, num_neighbours)
    nearest_neighbours_f = [nn for nn in nearest_neighbours_f[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    if set(nearest_neighbours_i)!=set(nearest_neighbours_f):
        print(nearest_neighbours_i)
        print(nearest_neighbours_f)
        print("**********************************")
        print("Atoms coordinating the metal have changed: {}".format(final_structure))
        print("**********************************")
        logs_to_check.append(final_structure)
        names2check.append(final_structure.split("/")[-1])
        return False
    else:
        # all the indices are the same for init and final, just check the bond length change
        for index in nearest_neighbours_i:
            if final_distances[index]>1.2*init_distances[index]:
                #print(index)
                print("Bond distance 20% increase for {}".format(log))
                logs_to_check.append(log)
                names2check.append(log.split("/")[-1])
                return False
    
    return True

Having flagged these optimizations these simulations were checked manually to determine whether the geometry was truly wrong.  